In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load the MNIST dataset
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Build the Variational Autoencoder (VAE) model
latent_dim = 2  # Latent space dimension

# Encoder
encoder_inputs = keras.Input(shape=(28, 28))
x = layers.Flatten()(encoder_inputs)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(128, activation="relu")(latent_inputs)
x = layers.Dense(28 * 28, activation="sigmoid")(x)
decoder_outputs = layers.Reshape((28, 28))(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# VAE model
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae = keras.Model(encoder_inputs, vae_outputs, name="vae")

# Define custom loss function for VAE
def vae_loss(encoder, decoder, original, reconstructed):
    original = tf.reshape(original, [-1, 28 * 28])  # Reshape original
    reconstructed = tf.reshape(reconstructed, [-1, 28 * 28])  # Reshape reconstructed
    reconstruction_loss = keras.losses.binary_crossentropy(original, reconstructed)
    reconstruction_loss *= 28 * 28
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
    return tf.reduce_mean(reconstruction_loss + kl_loss)

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda original, reconstructed: vae_loss(encoder, decoder, original, reconstructed))

# Train the VAE
vae.fit(x_train, x_train, epochs=10, batch_size=32)

# Generate samples from the trained VAE
random_samples = np.random.normal(0, 1, size=(16, latent_dim))
generated_images = decoder.predict(random_samples)

# You can now visualize the generated images or use them for your specific application.


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 9s 152us/sample - loss: 185.2920
Epoch 2/10
60000/60000 [==============================] - 7s 110us/sample - loss: 167.2100
Epoch 3/10
60000/60000 [==============================] - 7s 121us/sample - loss: 163.9949
Epoch 4/10
60000/60000 [==============================] - 7s 118us/sample - loss: 161.8808
Epoch 5/10
60000/60000 [==============================] - 6s 103us/sample - loss: 160.0927
Epoch 6/10
60000/60000 [==============================] - 8s 133us/sample - loss: 158.5483
Epoch 7/10
60000/60000 [==============================] - 6s 100us/sample - loss: 157.4660
Epoch 8/10
60000/60000 [==============================] - 8s 131us/sample - loss: 156.6327
Epoch 9/10
60000/60000 [==============================] - 7s 113us/sample - loss: 155.9539
Epoch 10/10
60000/60000 [==============================] - 8s 132us/sample - loss: 155.3016


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
